In [ ]:
import torch.nn as nn
import random
import torch
import numpy as np
import cv2
import torchvision.transforms as T
from PIL import Image
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import math
import os 

#### load net

In [ ]:
# # set device
# os.environ['CUDA_VISIBLE_DEVICES'] = '3'
# # set random seed
# seed = 777
# torch.manual_seed(seed)
# torch.cuda.manual_seed(seed)
# np.random.seed(seed)
# random.seed(seed)
# # set params; net: vgg, densenet        
# model = 'densenet' 
# # select and load an model
# if model == 'vgg19':
#     from models.vgg19 import Net 
# elif model == 'vgg16':
#     from models.vgg16 import Net 
# elif model == 'vgg16':
#     from models.vgg13 import Net 
# elif model == 'densenet':
#     from models.densenet import Net
# else:
#     print('model: {} is not support!'.format(model))
# net = Net()
# net = nn.DataParallel(net, device_ids=[0])
# net.load_state_dict(torch.load('outputs/best_{}_freeze.pth'.format(model))) 
# net.cuda()
# net.eval()
# # transform
# transform = T.Compose([
#     T.ToTensor(),
#     T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     T.Resize([600,600]),
# ])

In [ ]:
# find name by idx
def findbyidx(idx):
    name = None
    if idx == 0:
        name = 'drone'
    elif idx == 1:
        name = 'fighter-jet'
    elif idx == 2:
        name = 'helicopter'
    elif idx == 3:
        name = 'missile'
    elif idx == 4:
        name = 'passenger-plane' 
    else:
        name = 'rocket'
    return name

#### test image

In [ ]:
# # test image
# with open('data/planes/test.txt', 'r') as f:
#     lines = f.readlines()
# if not os.path.exists('errors'):
#     os.makedirs('errors')
# with open('errors/errors.txt', 'a+') as f:
#     for line in tqdm(lines):
#         im_path, gt = line.split(' ')
#         gt = int(gt)
#         im = cv2.imread(im_path)
#         if im is None:
#             im = np.asarray(Image.open(im_path).convert('RGB'))
#         im = transform(im).unsqueeze(0).cuda()
#         output = net(im)
#         _, idx = torch.max(output, dim=1)
#         idx = idx.item()
#         if idx != gt:
#             f.write('{} {} {}\n'.format(im_path, gt, idx))


#### visualization

In [ ]:
# # all to one
# with open('errors/errors.txt', 'r') as f:
#     lines = f.readlines()[:20]
# a = int(math.sqrt(len(lines)))+1
# n = 0
# fig, ax = plt.subplots(a,a, figsize=(4*a,4*a))
# for line in tqdm(lines):
#     im_path, gt, pred = line.split(' ')
#     gt, pred = int(gt), int(pred)
#     im = cv2.imread(im_path)
#     if im is None:
#         im = np.asarray(Image.open(im_path).convert('RGB'))
#     im = cv2.resize(im, (800,600))
#     row, col = int(n // a), int(n % a)
#     ax[row,col].imshow(im)
#     ax[row,col].set_title('GT: {}   Pred: {}\n{}'.format\
#         (findbyidx(gt), findbyidx(pred), im_path.replace('data/plane/', '')), color='red', fontsize=15)
#     ax[row,col].axis('off')
#     n += 1
# fig.tight_layout()
# if not os.path.exists('errors'):
#     os.makedirs('errors')
# plt.savefig('errors/demo.svg',format='svg')


In [ ]:
# devide by error preds
categories = ['drone', 'fighter-jet', 'helicopter', 'missile', 'passenger-plane', 'rocket']
n = {}
for category in categories:
    n[category] = 1
    categoty_dir = os.path.join('errors', category)
    if not os.path.exists(categoty_dir):
        os.makedirs(categoty_dir)
with open('errors/errors.txt', 'r') as f:
    lines = f.readlines()
for line in tqdm(lines):
    im_path, gt, pred = line.split(' ')
    gt_name = findbyidx(int(gt))
    pred_name = findbyidx(int(pred))
    im = cv2.imread(im_path)
    if im is None:
        im = np.asarray(Image.open(im_path).convert('RGB'))
    else:
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = cv2.resize(im, (800,600))
    plt.figure(figsize=(6,6))
    plt.imshow(im)
    plt.title('GT: {}   Pred: {}\n{}'.format\
        (gt_name, pred_name, im_path.replace('data/plane/', '')), color='red', fontsize=15)
    plt.axis('off')
    plt.savefig('errors/{}/{}.png'.format(gt_name, n[gt_name]),format='png')
    n[gt_name] += 1

#### outside image

In [ ]:
# # outside image
# for i in range(1, 13):
#     im_path = 'demo/{}.jpg'.format(i)
#     im = cv2.imread(im_path)
#     if im is None:
#         im = np.asarray(Image.open(im_path).convert('RGB'))
#     im = transform(im).unsqueeze(0).cuda()
#     output = net(im)
#     _, idx = torch.max(output, dim=1)
#     name = findbyidx(idx.item())
#     print(name)
